# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

In [1]:
from collections import Counter
import multiprocessing as mp
import csv
import math

In [2]:
path1 = "D:/FinUniver/Технологии обработки больших данных/Семинары/10_multiprocessing/10_multiprocessing_data/"

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [3]:
%%file count_letters.py
from collections import Counter

def count_letters(file):
    with open(file) as fp:
        text = fp.read().lower()
    return Counter(text)

Overwriting count_letters.py


In [4]:
from count_letters import count_letters

In [38]:
%%time
count_letters(path1 + "Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt")
count_letters(path1 + "Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt")

Wall time: 204 ms


Counter({'с': 50084,
         'п': 25652,
         'а': 73555,
         'и': 62030,
         'б': 16016,
         'о': 106740,
         ',': 26973,
         ' ': 182305,
         'ч': 16492,
         'т': 59813,
         'к': 30802,
         'л': 42328,
         'н': 60920,
         'г': 16174,
         'у': 27309,
         'в': 43700,
         'е': 80972,
         'й': 9747,
         'э': 3203,
         'р': 39784,
         'b': 25,
         'o': 104,
         'k': 16,
         's': 96,
         'c': 42,
         'a': 98,
         'f': 23,
         'e': 162,
         '.': 9864,
         'n': 114,
         't': 98,
         ':': 984,
         'h': 48,
         'p': 29,
         '/': 22,
         '\n': 8583,
         'u': 86,
         'r': 76,
         'd': 38,
         'v': 65,
         'i': 235,
         'y': 5,
         '_': 8,
         '-': 3558,
         '1': 384,
         '0': 110,
         '9': 100,
         '6': 271,
         'm': 54,
         'l': 46,
         'ж': 10552,
     

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

In [117]:
%%time
if __name__ == '__main__':
    files = [path1 + "Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt",
             path1 + "Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt"]

    with mp.Pool(processes=len(files)) as pool:
        counters = pool.map(count_letters, files)

['D:/FinUniver/Технологии обработки больших данных/Семинары/10_multiprocessing/10_multiprocessing_data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt', 'D:/FinUniver/Технологии обработки больших данных/Семинары/10_multiprocessing/10_multiprocessing_data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt']
Wall time: 491 ms


In [7]:
type(counters)

list

In [8]:
counters[0]

Counter({'с': 11507,
         'п': 5489,
         'а': 18236,
         'и': 13587,
         'б': 3980,
         'о': 23130,
         ',': 6372,
         ' ': 45076,
         'ч': 4113,
         'т': 14245,
         'к': 6744,
         'л': 9961,
         'н': 14240,
         'г': 3948,
         'у': 6044,
         'в': 9398,
         'е': 20054,
         'й': 2028,
         'э': 836,
         'р': 9482,
         'b': 220,
         'o': 377,
         'k': 21,
         's': 429,
         'c': 324,
         'a': 590,
         'f': 52,
         'e': 1200,
         '.': 2954,
         'n': 459,
         't': 332,
         ':': 212,
         'h': 227,
         'p': 100,
         '/': 20,
         '\n': 2734,
         'u': 285,
         'r': 308,
         'd': 192,
         'v': 87,
         'i': 369,
         'y': 8,
         '_': 4,
         '-': 900,
         '1': 46,
         '0': 22,
         '9': 36,
         '6': 42,
         'm': 401,
         'l': 571,
         'ж': 2297,
         'д

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

Фрагмент одного из файлов, которые должны получиться в результате:
```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [32]:
with open(path1 + "recipes_full.csv", 'r', encoding='utf-8') as fp:
    reader = csv.reader(fp)
    next(reader)
    row_count = sum(1 for row in reader)
print(row_count)

2231637


In [48]:
parts = 8
counts = 0
file_number = 0

for i in range(parts):
    with open(path1 + f"id_tag_nsteps_{i + 1}.csv", 'w', encoding='utf-8', newline='') as fw:
        writer = csv.writer(fw, delimiter=';')
        writer.writerow(['id', 'tag', 'n_steps'])

with open(path1 + "recipes_full.csv", 'r', encoding='utf-8') as fp:
    reader = csv.reader(fp)
    header = next(reader)
    
    for row in reader:
        line = [int(row[1]), row[5].replace('[', "").replace(']', "").replace("'", "").split(', '), int(row[6])]
        if counts % math.ceil(row_count / parts) == 0: 
            print(counts)
            file_number += 1
        with open(path1 + f"id_tag_nsteps_{file_number}.csv", 'a', encoding='utf-8', newline='') as fw:
            writer = csv.writer(fw, delimiter=';')
            for tag in line[1]:
                writer.writerow([line[0], tag, line[2]])
        counts += 1

0
278955
557910
836865
1115820
1394775
1673730
1952685


2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [19]:
def steps_aver(file: str) -> dict:
    tag_steps = dict()
    with open(file, 'r') as fp:
        reader = csv.reader(fp, delimiter=';')
        header = next(reader)
        for row in reader: 
            if row[1] not in tag_steps:
                tag_steps[row[1]] = [0, 0]
            tag_steps[row[1]][0] += int(row[2])
            tag_steps[row[1]][1] += 1
            
    tag_steps_aver = dict()
    for tag, steps in tag_steps.items():
        sums = steps[0]
        count = steps[1]
        tag_steps_aver[tag] = sums / count
    
    return tag_steps_aver

In [20]:
%%time
steps_aver(path1 + "id_tag_nsteps_1.csv")

Wall time: 2.1 s


{'mexican': 5.302503052503052,
 'healthy-2': 6.328114004222378,
 'orange-roughy': 3.4451697127937337,
 'chicken-thighs-legs': 4.184877440797673,
 'freezer': 3.9144921718185466,
 'whitefish': 3.5132206328565236,
 'pork-sausage': 4.285714285714286,
 'brunch': 6.900977198697069,
 'ham-and-bean-soup': 3.5126970227670755,
 'colombian': 3.5766456266907123,
 'savory-pies': 4.294961081523965,
 'refrigerator': 4.74200503054258,
 'australian': 4.25384024577573,
 'served-cold': 4.940726577437858,
 'spaghetti': 4.1286239281339325,
 'passover': 3.6305130513051305,
 'quick-breads': 4.969485903814262,
 'californian': 3.7421907538525616,
 'namibian': 3.495145631067961,
 'candy': 4.302595893064703,
 'independence-day': 4.143187066974596,
 'baking': 3.5864793678665494,
 'pennsylvania-dutch': 3.536219459922846,
 'weeknight': 7.406489318823056,
 '60-minutes-or-less': 9.344522810463351,
 'time-to-make': 9.248938170735661,
 'course': 9.242875343659895,
 'cuisine': 9.133115942028985,
 'preparation': 9.293666

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [3]:
%%file steps_aver_func.py
import csv

def steps_aver(file: str) -> dict:
    tag_steps = dict()
    with open(file, 'r') as fp:
        reader = csv.reader(fp, delimiter=';')
        header = next(reader)
        for row in reader: 
            if row[1] not in tag_steps:
                tag_steps[row[1]] = [0, 0]
            tag_steps[row[1]][0] += int(row[2])
            tag_steps[row[1]][1] += 1
    
#     tag_steps_aver = dict()
#     for tag, steps in tag_steps.items():
#         sums = steps[0]
#         count = steps[1]
#         tag_steps_aver[tag] = sums / count
    
    return tag_steps

def unite_steps_aver(all_files_results: list) -> dict:
    all_steps_aver_buf = dict()
    
    for result in all_files_results:
        for tag, steps in result.items():
            if tag not in all_steps_aver_buf:
                all_steps_aver_buf[tag] = [0, 0]
            all_steps_aver_buf[tag][0] += steps[0]
            all_steps_aver_buf[tag][1] += steps[1]
     
    all_steps_aver_dict = dict()
    for tag, steps in all_steps_aver_buf.items():
        sums = steps[0]
        count = steps[1]
        all_steps_aver_dict[tag] = sums / count
    
    return all_steps_aver_dict

Overwriting steps_aver_func.py


In [4]:
def all_steps_aver(files: list) -> dict:
    all_files_results = list()
    
    for file in files:
        all_files_results.append(steps_aver(file))
        
    return unite_steps_aver(all_files_results)

In [5]:
from steps_aver_func import steps_aver, unite_steps_aver

In [18]:
%%time
files = [path1 + f"id_tag_nsteps_{i}.csv" for i in range(1, 9)]
all_steps_aver(files)

Wall time: 17 s


{'mexican': 5.303645328113331,
 'healthy-2': 6.384524752622533,
 'orange-roughy': 3.513571517526461,
 'chicken-thighs-legs': 4.145724762032948,
 'freezer': 4.033609555792238,
 'whitefish': 3.514269832464186,
 'pork-sausage': 4.255861221720788,
 'brunch': 6.873000781695458,
 'ham-and-bean-soup': 3.5084218879581752,
 'colombian': 3.5361185007534726,
 'savory-pies': 4.298318609599992,
 'refrigerator': 4.702481105672773,
 'australian': 4.218506716782309,
 'served-cold': 4.91160803593937,
 'spaghetti': 4.082097304289028,
 'passover': 3.6585278659940026,
 'quick-breads': 5.05880995306798,
 'californian': 3.7411401628000798,
 'namibian': 3.5045866139970494,
 'candy': 4.229798788252008,
 'independence-day': 4.1060809377110665,
 'baking': 3.6306153248665325,
 'pennsylvania-dutch': 3.5469561200938657,
 'weeknight': 7.4135187359635015,
 '60-minutes-or-less': 9.413529516520915,
 'time-to-make': 9.27861745373393,
 'course': 9.274718384210226,
 'cuisine': 9.170586594125286,
 'preparation': 9.2935199

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [6]:
%%time
if __name__ == '__main__':
    files = [path1 + f"id_tag_nsteps_{i}.csv" for i in range(1, 9)]
    all_files_results = list()
    
    with mp.Pool(processes=len(files)) as pool:
        all_files_results = pool.map(steps_aver, files)
        
    result = unite_steps_aver(all_files_results)
result  

Wall time: 8.24 s


{'mexican': 5.302344316442439,
 'healthy-2': 6.384162244806188,
 'orange-roughy': 3.513425052701653,
 'chicken-thighs-legs': 4.145581465931509,
 'freezer': 4.033042234819468,
 'whitefish': 3.514734127201888,
 'pork-sausage': 4.256068444090729,
 'brunch': 6.871661962657403,
 'ham-and-bean-soup': 3.508423254789694,
 'colombian': 3.5359842260926717,
 'savory-pies': 4.298328243879716,
 'refrigerator': 4.702350782137551,
 'australian': 4.218603314493725,
 'served-cold': 4.911663673979233,
 'spaghetti': 4.0825152293208475,
 'passover': 3.658676110051757,
 'quick-breads': 5.058895036887995,
 'californian': 3.74143203627544,
 'namibian': 3.5042895887529752,
 'candy': 4.229612689762553,
 'independence-day': 4.10637159533074,
 'baking': 3.6306821245618766,
 'pennsylvania-dutch': 3.5471966710468683,
 'weeknight': 7.413649806241077,
 '60-minutes-or-less': 9.413654300607185,
 'time-to-make': 9.278520775418526,
 'course': 9.274676657987765,
 'cuisine': 9.17006030766978,
 'preparation': 9.29343297028

5. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди `multiprocessing.queue` передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [36]:
%%file worker.py
import multiprocessing as mp
from steps_aver_func import steps_aver, unite_steps_aver
def worker(ins, outs):
    outs.put(steps_aver(ins.get()))

Overwriting worker.py


In [37]:
from worker import worker

In [38]:
%%time
if __name__ == '__main__':
    number_of_processes = mp.cpu_count() // 2
    files = [path1 + f"id_tag_nsteps_{i}.csv" for i in range(1, 9)]

    queue_files = mp.Queue()
    queue_results = mp.Queue()

    for file in files:
        queue_files.put(file)

#     with mp.Pool(processes=number_of_processes) as pool:
#         pool.map(worker, (queue_files, queue_results))
    for i in range(queue_files.qsize()):
        mp.Process(target=worker, args=(queue_files, queue_results)).start()

Wall time: 173 ms


In [51]:
queue_results.qsize()

8

In [52]:
all_files_results = [queue_results.get() for _ in range(queue_results.qsize())]
unite_steps_aver(all_files_results)

{'mexican': 5.302344316442439,
 'healthy-2': 6.384162244806188,
 'orange-roughy': 3.513425052701653,
 'chicken-thighs-legs': 4.145581465931509,
 'freezer': 4.033042234819468,
 'whitefish': 3.514734127201888,
 'pork-sausage': 4.256068444090729,
 'brunch': 6.871661962657403,
 'ham-and-bean-soup': 3.508423254789694,
 'colombian': 3.5359842260926717,
 'savory-pies': 4.298328243879716,
 'refrigerator': 4.702350782137551,
 'australian': 4.218603314493725,
 'served-cold': 4.911663673979233,
 'spaghetti': 4.0825152293208475,
 'passover': 3.658676110051757,
 'quick-breads': 5.058895036887995,
 'californian': 3.74143203627544,
 'namibian': 3.5042895887529752,
 'candy': 4.229612689762553,
 'independence-day': 4.10637159533074,
 'baking': 3.6306821245618766,
 'pennsylvania-dutch': 3.5471966710468683,
 'weeknight': 7.413649806241077,
 '60-minutes-or-less': 9.413654300607185,
 'time-to-make': 9.278520775418526,
 'course': 9.274676657987765,
 'cuisine': 9.17006030766978,
 'preparation': 9.29343297028